## Librerie

In [1]:
from scipy.optimize import minimize, basinhopping
import numpy as np
import pandas as pd

## Parametri problema

In [2]:
# Consumi famiglia 1, famiglia 2, ...
consumi = np.array([1500, 5700, 3400, 300])

# Produzioni ricetta1, ricetta2, ...
produzioni = np.array([3000, 6300, 1200])

# Ricette
#           | Ricetta1 | Ricetta2 | ...
# --------------------------------------
# Famiglia1 |          |          |
# Famiglia2 |          |          |
# ...
ricette = np.array([
    0.25, 0.2, 0.3,
    0.43, 0.5, 0.35,
    0.3, 0.27, 0.35,
    0.02, 0.03,  0
])

# Composizioni ricette per famiglia
#           | Materiale1 | Materiale2 | ...
# --------------------------------------
# Famiglia1 |            |            |
# Famiglia2 |            |            |
# ...
composizioni_famiglia = np.array([
    0.58, 0.42, 0,
    1, 0, 0,
    0, 1, 0,
    0, 0, 1
])

# Range ammissibile percentuale materiale per ricetta
#           | Materiale1        | Materiale2 | ...
# ------------------------------------------------
# Ricetta1  | (val att. ,range) |            |
# Ricetta2  |                   |            |
# ...
range_ric_mat = np.array([
    [(0.58, 0.01), (0.396, 0.003), (0.024, 0.001)], # ricetta 0
    [(0.625, 0.005), (None, None), (None, None)],
    [(0.62, 0.01), (None, None), (None, None)],
])

In [3]:
%run ./importazione.ipynb

(10,)
(28,)
(28, 10)
(28, 13)
(10, 13)
(10, 13)
(10, 13)


In [4]:
print('Verifica dimensioni matrici')
print(f'Consumi (num. famiglie)): {consumi.shape}')
print(f'Produzioni (num. ricette): {produzioni.shape}')
print(f'Ricette (famiglie x ricette): {ricette.shape}')
print(f'Composizioni (famiglie x materiali): {composizioni_famiglia.shape}')
print(f'Range (ricette x materiali): {range_ric_mat.shape}')

Verifica dimensioni matrici
Consumi (num. famiglie)): (28,)
Produzioni (num. ricette): (10,)
Ricette (famiglie x ricette): (280,)
Composizioni (famiglie x materiali): (364,)
Range (ricette x materiali): (10, 13)


## Accrocchio per compattare ricetta

In [5]:
POSITIONS_COMP = []
RICETTA_COMP = []

for index, value in enumerate(ricette):
    if value != 0:
        POSITIONS_COMP.append(index)
        RICETTA_COMP.append(value)

def rebuild_ricetta(ricetta_comp):
    rebuild = np.zeros(len(ricette))
    for index, pos in enumerate(POSITIONS_COMP):
        rebuild[pos] = ricetta_comp[index]
    return rebuild

print(rebuild_ricetta(RICETTA_COMP)[0:20])
print(ricette[0:20])
print(len(RICETTA_COMP))

[0.    0.029 0.    0.    0.    0.    0.    0.    0.    0.    0.455 0.48
 0.03  0.475 0.    0.    0.    0.    0.005 0.   ]
[0.    0.029 0.    0.    0.    0.    0.    0.    0.    0.    0.455 0.48
 0.03  0.475 0.    0.    0.    0.    0.005 0.   ]
76


## Calcolo resa globale

In [6]:
tot_consumi = np.sum(consumi)
tot_produzioni = np.sum(produzioni)
resa_globale = tot_consumi / tot_produzioni
f'{resa_globale=}'

'resa_globale=np.float64(1.0389346559077535)'

## Funzioni di calcolo

In [7]:
# Calcola matrice consumi moltiplicando matrice ricetta in input per produzioni
def calc_mat_consumi(ricetta):
    ricetta = rebuild_ricetta(ricetta) #REBUILD
    return ricetta.reshape(-1, len(produzioni)) * produzioni #/ resa_globale

In [8]:
# Calcola vettore consumi complessivi partendo da produzioni iniziali e matrice consumi
def calc_tot_consumi(matrice_consumi):
    return np.sum(matrice_consumi, axis=1)

In [9]:
# Calcolo errore su totali consumi
def calc_err_totali(ricetta):
    matrice_consumi = calc_mat_consumi(ricetta)
    tot_consumi = calc_tot_consumi(matrice_consumi)
    tot_err = np.sqrt(np.sum(np.square(tot_consumi-consumi)))/1000
    return tot_err

In [10]:
# Calcola rese per famiglia 
# (consumi per famiglia / produzione)
def calc_tot_resa(matrice_consumi):
    return np.sum(matrice_consumi, axis=0)/produzioni

In [11]:
# Calcolo errore su percentuali prod. effettive rispetto a resa totale
def calc_error_resa(ricetta):
    matrice_consumi = calc_mat_consumi(ricetta)
    tot_resa = calc_tot_resa(matrice_consumi)
    return np.sum(np.square(tot_resa - resa_globale))

In [12]:
# Percentuali di ciascun materiale in una ricetta
def perc_mat(ricetta, id_ric):
    cons_fam = calc_mat_consumi(ricetta)
    compos = composizioni_famiglia.reshape(len(consumi), -1)
    cons_ricetta0 = np.vstack(cons_fam[:,id_ric])
    cons_materiali_ricetta0 = cons_ricetta0 * compos
    tot_cons_ricetta0 = np.sum(cons_materiali_ricetta0, axis=0)
    tot_perc_ricetta0 = tot_cons_ricetta0 / np.sum(tot_cons_ricetta0) if np.sum(tot_cons_ricetta0) != 0 else np.zeros((len(tot_cons_ricetta0),1))
    return tot_perc_ricetta0

In [13]:
# Funzione errore percentuale materiale (obiettivo: >=0)
def err_perc_mat(ricetta, id_ric, id_mat, expected_val, expected_error):
    return expected_error - np.abs(perc_mat(ricetta, id_ric=id_ric)[id_mat] - expected_val)

## Ottimizzazione

In [14]:
%%time

constraints = [
    {'type': 'eq', 'fun': calc_err_totali},
]

for id_ric, ric in enumerate(range_ric_mat):
    for id_mat, mat in enumerate(ric):
        if not np.isnan(mat[0]):
            constr = {'type': 'ineq', 'fun': err_perc_mat, 'args': (id_ric, id_mat, mat[0], mat[1])}
            constraints.append(constr)

# constraints = (
#     {'type': 'eq', 'fun': calc_err_totali},
#     {'type': 'ineq', 'fun': err_perc_mat, 'args': (0, 0, 0.58, 0.01)},
#     {'type': 'ineq', 'fun': err_perc_mat, 'args': (0, 1, 0.396, 0.003)},
#     {'type': 'ineq', 'fun': err_perc_mat, 'args': (0, 2, 0.024, 0.001)},
#     {'type': 'ineq', 'fun': err_perc_mat, 'args': (1, 0, 0.625, 0.005)},
#     # {'type': 'ineq', 'fun': err_perc_mat, 'args': (2, 0, 0.62, 0.01)},
# )

#DEBUG
print(len(constraints))
# constraints = constraints[0:40] #DEBUG 40 ok, oltre boom

bounds = list(( (0, 1) for x in range(len(RICETTA_COMP) )))

count_iterations = 0
def callback(x, y=None):
    global count_iterations
    print(f'{count_iterations} - {calc_err_totali(x)}')
    count_iterations += 1

# Minimizzazione con TRUST-CONSTR
res = minimize(
    calc_error_resa, 
    RICETTA_COMP, 
    method='trust-constr', 
    # method='SLSQP',
    constraints=constraints,
    bounds=bounds,
    callback=callback,
    options={'disp': True, 'maxiter':100}
)

# Basin hopping con TRUST_CONSTR
# res = basinhopping(
    
#     calc_error_resa, 
#     RICETTA_COMP,
    
#     stepsize=0.05,
#     niter=50,
    
#     minimizer_kwargs = {
#         "method":'trust-constr',
#         "constraints":constraints,
#         "bounds":bounds,
#         "callback":callback,
#         "options":{'disp': True, 'maxiter':200}
#     }
    
# )


res.x = res.x / resa_globale

print(res)

# res = basinhopping(
#     calc_error_resa, 
#     RICETTA_COMP,
#     stepsize=0.05,
#     niter=10,
    
#     minimizer_kwargs = {
#         "method":'SLSQP',
#         "constraints":constraints,
#         "bounds":bounds,
#         "callback":callback,
#         "options":{'disp': True, 'maxiter':20}
#     }
# )
# print(res)

# for i in range(10):
    
#     res = minimize(
#         calc_error_resa, 
#         res.x, 
#         method='trust-constr', 
#         # method='SLSQP',
#         constraints=constraints,
#         bounds=bounds,
#         options={'disp': True, 'maxiter':10}
#     )
        
#     res = minimize(
#         calc_error_resa, 
#         res.x,
#         method='SLSQP',
#         constraints=constraints,
#         bounds=bounds,
#         callback=callback,
#         options={'disp': True, 'maxiter':10}
#     )

# print(res)


131
0 - 18234.605238100565


C:\Users\mario.piccinelli\Documents\Progetti\Almag-Spalmatura_prod\.venv\Lib\site-packages\scipy\optimize\_differentiable_functions.py:552: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(delta_x, delta_g)


1 - 3075.6844397582718
2 - 3075.6844397582718
3 - 3075.6844397582718
4 - 3075.6844397582718
5 - 3075.6844397582718
6 - 3004.1754981039867
7 - 2944.475119484692
8 - 2944.475119484692
9 - 2944.475119484692
10 - 2919.6448207366966
11 - 2919.6448207366966
12 - 2896.2679238380038
13 - 2896.2679238380038
14 - 2880.346098881928
15 - 2857.9427006814976
16 - 2857.9427006814976
17 - 2843.265502693526
18 - 2821.7202070222856
19 - 2814.803353931216
20 - 2791.227720900386
21 - 2785.054437719663
22 - 2761.685690172319
23 - 2755.584075012221
24 - 2733.0459666240336
25 - 2726.546547111966
26 - 2705.054257524563
27 - 2698.049771569668
28 - 2677.6408182215746
29 - 2670.1732199902744
30 - 2650.8794875379263
31 - 2642.9373664181717
32 - 2624.6404606001765
33 - 2616.309790634198
34 - 2599.002053149912
35 - 2590.0881419624293
36 - 2573.8918164576853
37 - 2564.44599024304
38 - 2549.2948733654066
39 - 2539.3284355727237
40 - 2525.2078337916223
41 - 2514.824574643819
42 - 2501.45960395663
43 - 2490.78159530715

KeyboardInterrupt: 

In [15]:
# Esperimento
# Proviamo a escludere un constraint alla volta e vediamo se troviamo una situazione
# in cui il problema converge.

# for i in range(1, len(constraints)):
    
#     print(i)
    
#     const_red = constraints[0:i] + constraints[i+1:]

#     res = minimize(
#         calc_error_resa, 
#         ricette, 
#         method='SLSQP',
#         constraints=const_red,
#         bounds=bounds,
#         options={'disp': False, 'maxiter':100}
#     )

#     print(f'{res.success} - {res.message} ({res.nit} iterations)')


## Verifiche

In [16]:
pd.options.display.float_format = '{:.2f}'.format

In [17]:
# res.x = res.x / resa_globale
result = rebuild_ricetta(res.x)

NameError: name 'res' is not defined

In [ ]:
print("Percentuali aggiustate (in %)")
print(np.sum(result.reshape(len(consumi), len(produzioni))*100, axis=0))
pd.DataFrame(
    result.reshape(len(consumi), len(produzioni))*100
)

In [ ]:
print("Matrice consumi")
pd.DataFrame(
    calc_mat_consumi(res.x)
)

In [ ]:
print('Verifica totale consumi')
pd.DataFrame(
    [calc_tot_consumi(calc_mat_consumi(res.x))]
)

In [ ]:
print('Verifica errore consumi')
pd.DataFrame([calc_tot_consumi(calc_mat_consumi(res.x))*resa_globale - consumi])

In [ ]:
# np.sum(calc_mat_consumi(res.x)*resa_globale, axis=1) / consumi * 100

In [ ]:
print(f'Verifica rese (resa globale: {resa_globale:.2f})')
pd.DataFrame(
    [calc_tot_resa(calc_mat_consumi(res.x))]
)

In [ ]:
print(f'Verifica errore rese (resa globale: {resa_globale:.2f})')
pd.DataFrame(
    [calc_tot_resa(calc_mat_consumi(res.x)) - resa_globale]
)

### Errori percentuali materiali

In [ ]:
def print_err_mat(id_ricetta, ricetta):
    return pd.DataFrame([
        perc_mat(RICETTA_COMP, id_ricetta) * 100,
        map(lambda x: x[0]*100, list(range_ric_mat[id_ricetta])),
        perc_mat(ricetta, id_ricetta) * 100
    ])

In [ ]:
print('Percentuali materiali ricetta 0')
print_err_mat(0, res.x)

In [ ]:
print('Percentuali materiali ricetta 1')
print_err_mat(1, res.x)

In [ ]:
print('Percentuali materiali ricetta 2')
print_err_mat(2, res.x)

### Funzioni errore

In [ ]:
for index,c in enumerate(constraints):
    # print(c)
    if 'args' in c:
        print(f"{index} - {c['fun'](res.x, *c['args']):.4f} (ric:{c['args'][0]} mat:{c['args'][1]})")
    else:
        print(c['fun'](res.x))

# ?

In [ ]:
r = rebuild_ricetta(res.x).reshape(len(consumi), len(produzioni))
print(np.sum(r, axis=0))
print(np.sum(r*produzioni, axis=1))
print(np.sum(r*produzioni, axis=1) - consumi)
print(np.sum(np.sum(r*produzioni, axis=1) - consumi))

In [ ]:
#pd.DataFrame(rebuild_ricetta(res.x)).to_csv('trust_constr_5k_iter.csv')